### На основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)


In [44]:
import pandas as pd
import json
import tarfile
import numpy as np
import nltk
from gensim.models import Word2Vec, FastText
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
# распаковка архива
tar = tarfile.open('gazeta_jsonl_v2.tar.gz', 'r:gz')
tar.extractall()
tar.close()

In [12]:

data = []
with open('gazeta_train.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        record = json.loads(line)
        data.append(record)
data = pd.DataFrame(data)

In [37]:
# берем только тексты
texts = data['text'].tolist()

In [38]:
# токенизация
corpus = []
for doc in texts:
    tokens = nltk.word_tokenize(doc.lower())
    corpus.append(tokens)

In [39]:
# обучение
word2vec = Word2Vec(corpus, vector_size =300, window=5, min_count=5, workers=8)


In [25]:
# обучение
fasttext = FastText(corpus, vector_size=300, window=5, min_count=5, workers=8)

In [34]:
def find_similar_articles(query, num_results=5, model_type="word2vec"):
    if model_type == "word2vec":
        # обучение модели word2vec
        model = word2vec
    elif model_type == "fasttext":
        # обучение модели fasttext
        model = fasttext

    # токенизация запроса
    query_tokens = nltk.word_tokenize(query.lower())

    # получение векторного представления запроса
    query_vec = sum([model.wv[token] for token in query_tokens]) / len(query_tokens)

    # получение векторного представления каждой статьи
    article_vecs = []
    for article in data['text']:
        article_tokens = nltk.word_tokenize(article.lower())
        article_vec = sum([model.wv[token] for token in article_tokens]) / len(article_tokens)
        article_vecs.append(article_vec)

    # вычисление косинусного расстояния между вектором запроса и векторами
    similarities = cosine_similarity([query_vec], article_vecs)[0]

    # получение индексов наиболее похожих статей
    most_similar = similarities.argsort()[::-1][:num_results]

    # вывод наиболее похожих статей
    for i in most_similar:
        print(data.iloc[i]['title'])

In [53]:
find_similar_articles('Государственная дума', num_results=5, model_type='fasttext')

[{'text': 'Председатель комитета по конституционному законодательству и госстроительству Владимир Плигин и его первый зам Александр Москалец (оба — единороссы) внесли в Госдуму поправки, предлагающие изменить порядок назначения сенаторов. Депутаты предлагают установить, что член Совета федерации должен приступать к исполнению своих обязанностям через десять дней после вступления в силу решения о его избрании или назначении. Соответствующие поправки вносятся в ст. 2 президентского закона «О внесении изменений в отдельные законодательные акты РФ в связи с изменением порядка формирования Совета федерации». Этот документ был принят Госдумой еще в 2009 году, однако вступит в силу только с 1 января следующего года. Президентский проект вносил изменения в закон «О порядке формирования Совета федерации». В частности, документом устанавливалось, что полномочия нового члена верхней палаты должны начинаться «со дня его избрания (назначения) соответствующим органом государственной власти субъекта»

In [50]:
find_similar_articles('Государственная дума', num_results=5, model_type='word2vec')

[{'text': 'Конституционный суд (КС) России в четверг зафиксировал право субъектов РФ самостоятельно определять названия должностей и круг полномочий региональных чиновников. Прояснить ситуацию судей попросили депутаты Законодательного собрания Камчатского края, у которых возникли разногласия с региональной прокуратурой из-за перечня должностей краевых чиновников. В 2008 году ЗаКС Камчатского края утвердил региональный закон «О государственной гражданской службе», которым, кроме всего прочего, учреждался реестр должностей краевых чиновников. Спустя некоторое время протест по поводу принятого закона заявила камчатская прокуратура, усмотревшая противоречие с федеральным законодательством. «Реестр должностей государственной гражданской службы субъекта РФ утверждается… с учетом положений настоящего Федерального закона, а также структуры государственных органов, наименований, категорий и групп должностей государственной гражданской службы РФ, установленных Реестром должностей федеральной гос